Cells that must be completed to receive marks are labelled like this:

`# -- GRADED CELL (1 mark) - complete this cell --`

Some graded cells are code cells, in which you must complete the code to solve a problem. Other graded cells are markdown cells, in which you must write your answers to short-answer questions. 

You will see the following text in graded code cells:

```
# YOUR CODE HERE
raise NotImplementedError()
```

***You must remove the `raise NotImplementedError()` line from the cell, and replace it with your solution.***


Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and student ID number below:

In [ ]:
NAME = "Jiayu Wang"
ID = "1039580"

# COMP90014 Assignment 1
### Semester 2, 2021

This assignment should be completed by each student individually. Make sure you read this entire document, and ask for help if anything is not clear. Any changes or clarifications to this document will be announced via the LMS.

Please make sure you review the University's rules on academic honesty and plagiarism: https://academichonesty.unimelb.edu.au/

Do not copy any code from other students or from the internet. This is considered plagiarism.

To complete the assignment, finish the tasks in this notebook.

The tasks are a combination of writing your own code, interpreting the results and answering related short-answer questions.

In some cases, we have provided test input and test output that you can use to try out your solutions. These tests are just samples and are **not** exhaustive - they may warn you if you've made a mistake, but they are not guaranteed to. It's up to you to decide whether your code is correct.

### Completing the assignment

You will need to finish the tasks in this notebook.

You must not call functions from external packages that implement parts of the algorithms we have asked you to write.

The tasks are a combination of writing your own implementations of algorithms we've discussed in lectures and interpreting the results in short answer format.

In some case, we have provided test input and test output that you can use to try out your solutions. These tests are just samples and are not exhaustive - they may warn you if you've made a mistake, but they are not guaranteed to. It's up to you to decide whether your code is correct.

### Submitting your work

Your completed notebook file containing all your answers must be turned in via LMS in `.ipynb` format.
You must also submit a file in `html` format with the output cleared.
You can do this by using the `Clear all output` option in the menu.


### Marking

Only modify graded cells. If you want to write a helper function, do it in a graded cell.

Word limits, where stated, will be strictly enforced. Answers exceeding the limit **will not be marked**.

No marks are allocated to commenting in your code. We do however, encourage efficient and well commented code.


### Background and data
During this assignment we will analyse the relationship between 17 organisms. 
Understanding the genetic and evolutionary relationship between organisms is a core task in bioinformatics. 
This type of phylogenetic analysis can be performed by comparing the sequence differences of a particular gene between the organisms.
All organisms must possess the gene in question for comparison to be possible.  

As all organisms possess genes to make ribosomes, these will be used in this assignment.
A 100bp segment of the small ribosomal subunit gene has been extracted from each of these 17 organisms, and will act as our sequences for comparison.

In the first section, we will calculate distances between these organisms and visualise.
In the second section, we will use clustering algorithms to group organisms.

At each step of the process you will be asked to fill out short answer questions to demonstrate your understanding of the concepts.

### Import packages
If you are using jupyter lab online, all packages will be available. If you are running this on your local computer, you may need to install some packages.

In [ ]:
import numpy as np
import scipy
import pandas as pd
import sklearn
import seaborn
import matplotlib.pyplot as plt
%matplotlib inline

### Importing data
To begin, we need to understand the data.
The ribosome genes are provided in a .fasta file called 'ribosome_genes.fasta'. You can have a look if you like.
These genes will be imported as classes (RibosomeGene). 

Each RibosomeGene object has a name, accession, sequence and length.
You can access these properties using '.' syntax. (see below). 

Try to think of each gene as something physical, rather than code.
In real life, each gene has a .length, its organism has a .name, and it has a .sequence. We can write code in this way too. 

we will import these into **ribosome_genes**, which is a list of our genes.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from utilities import import_16s_sequences
ribosome_genes = import_16s_sequences()

In [ ]:
print('{:<20}{:<30}{:<15}{:<10}'.format(
    'gene.accession',
    'gene.name',
    'gene.length',
    'gene.sequence'))

for gene in ribosome_genes:
    print('{:<20}{:<30}{:<15}{:<10}'.format(
        gene.accession,
        gene.name[:27],
        gene.length,
        gene.sequence[:8] + '...'))

# SECTION 1: PAIRWISE DISTANCES

To be able to compare organisms via their sequences, we need a way to measure their difference as a distance. 

## TASK 1: K-mer distance

The kmer distance between two sequences is defined here as the total number of k-mers that are unique to either sequence.<br>

i.e. If seq1 has 3 unique kmers not found in seq2 (copy number difference also matters), and seq2 has 2 unique kmers, the kmer distance in this assignment is 5.  

Write a function that calculates a dictionary of k-mers (for k = any number) and their counts for each gene sequence.

In [ ]:
# -- GRADED CELL (2 marks) - complete this cell --

def create_kmer_dictionary(seq, k):
    '''
    Write a function which stores kmers, and their counts from a given input seq. 
    The kmer size == k 
    '''
    dic = {}
    length = len(seq)
    if k <= length and k != 0:
        
        for a in range(0,length - k + 1):
            if seq[a:a+k] in dic:
                dic[seq[a:a+k]] += 1
            else:
                dic[seq[a:a+k]] = 1
        
    return dic
    

In [ ]:
assert create_kmer_dictionary("AGAATATAACACACAA", 1) == {'A': 10, 'G': 1, 'T': 2, 'C': 3}

In [ ]:
# -- GRADED CELL (2 marks) - complete this cell --

def calculate_total_unique_kmers(kmers1, kmers2):
    '''
    Given two kmer occurance dictionaries, compute the sum of
    the differences in kmer frequencies.
    '''
    unique_kmers = 0
    for kmer in kmers1:
        if kmer in kmers2:
            if kmers1[kmer] != kmers2[kmer]:
                unique_kmers += (abs(kmers1[kmer] - kmers2[kmer]))
            else:
                pass
        else:
            unique_kmers += kmers1[kmer]
    for kmer in kmers2:
        if kmer in kmers1:
            continue
        else:
            unique_kmers += kmers2[kmer]
    
    return unique_kmers


In [ ]:
assert calculate_total_unique_kmers(
    {'A': 10, 'G': 1, 'T': 2, 'C': 3},
    {'A': 1, 'G': 10, 'T': 1, 'C': 3}) == 19

Write a function that uses the above two functions to calcualte the k-mer distance of two sequences

In [ ]:
# -- GRADED CELL (2 marks) - complete this cell --

def kmer_distance(seq1, seq2, k):
    '''
    For two input sequences, find their kmer distance.
    Use the create_kmer_dictionary and calculate_total_unique_kmers
    functions you have written previously. 
    '''
    dic1 = create_kmer_dictionary(seq1, k)
    dic2 = create_kmer_dictionary(seq2, k)
    
    distance =  calculate_total_unique_kmers(dic1, dic2)
    
    return distance


Let's check our function. We can use the first two entries in the 'ribosome_genes' list.
If implemented correctly, the following should return 86.

In [ ]:
print(kmer_distance(ribosome_genes[0].sequence,
                    ribosome_genes[1].sequence, 8))

assert kmer_distance(
    ribosome_genes[0].sequence,
    ribosome_genes[1].sequence,
    8) == 86

## TASK 2: Smith-Waterman alignment
Another way to compare the similarity of two sequences is through alignment.
The alignment score of two sequences will be high when they are similar, and low when they are distinct. 

Write a function that initialises and returns a scoregrid for two sequences.

Keep in mind the matrix must be 1 element larger than the sequence lengths. Consider whether indel scores for the first row and column need to be filled in.

In [ ]:
# -- GRADED CELL (1 mark) - complete this cell --

import numpy as np

def init_scoregrid(seq1, seq2):
    '''
    Given two sequences, initialise a scoregrid to be used for smith-waterman alignment.
    '''
    
    X = len(seq1) + 1
    Y = len(seq2) + 1
    scoregrid = np.zeros((X,Y), int)
    
    return scoregrid

In [ ]:
# Print the initialised scoregrid
print(init_scoregrid('croissant', 'crescent'))


Write a function that populates the scoregrid. It accepts two sequences, a scoregrid and match/mismatch and indel scores. 

In [ ]:
# -- GRADED CELL (2 marks) - complete this cell --

def calculate_scoregrid(seq1,
                        seq2,
                        scoregrid,
                        match_score=1,
                        mismatch_score=-4,
                        indel_score=-4):
    '''
        Given two sequences and an initialised scoregrid,
        fill in the cumulative alignment scores for each cell.
    '''
    
    X = len(seq1) + 1
    Y = len(seq2) + 1
    for x in range(1,X):
        for y in range(1,Y):
            if seq1[x-1] == seq2[y-1]: 
                diag = match_score
            else:
                diag = mismatch_score
            
            scoregrid[x,y] = max(scoregrid[x-1,y-1] + diag,
                                scoregrid[x-1,y] + indel_score, 
                                scoregrid[x,y-1] + indel_score,
                                0)
    
    return scoregrid

In [ ]:
scoregrid = init_scoregrid('cello', 'helllo')
print(calculate_scoregrid('cello', 'helllo', scoregrid))

The last thing we need is to get the alignment score from the scoregrid. 
Write a function that returns an alignment score (Smith-Waterman style, local alignment) from a scoregrid

In [ ]:
# -- GRADED CELL (1 marks) - complete this cell --

def report_alignment_score(scoregrid):
    ''' 
        Given a completed scoregrid, return the smith-waterman alignment score.
    '''
    sw_alignment_score = np.max(scoregrid)
    
    return sw_alignment_score

Ok! now we're ready to put it all together. <br>
Fill in the function below with the three functions you wrote to calculate the alignment score of two sequences

In [ ]:
# -- GRADED CELL (1 marks) - complete this cell --

def smith_waterman(seq1, seq2):
    '''
        Given two sequences, perform smith-waterman
        alignment score calculation. Use the init_scoregrid,
        calculate_scoregrid, and report_alignment_score functions
        you have written.
    '''
    scoregrid = init_scoregrid(seq1, seq2)
    scoregrid = calculate_scoregrid(seq1,
                        seq2,
                        scoregrid,
                        match_score=1,
                        mismatch_score=-4,
                        indel_score=-4)
    alignment_score = report_alignment_score(scoregrid)
    return alignment_score


## TASK 3: Pairwise distances

We can now calculate the distance of two sequences using *k*-mer distance and the Smith-Waterman alignment score. 
Let's use these two methods to calculate the pairwise distance of our genes. 

Write a function which initialises and returns a distance matrix from a list of genes. 
Use a nested dictionary, where keys are `gene.accession`, and the values are all 0 for now. 

In [ ]:
# -- GRADED CELL (1 mark) - complete this cell --
def init_distance_matrix(genes):
    '''
        For our list of genes, initialise a pairwise-distance matrix. Use accessions to denote genes. 
    '''
    distance_matrix = {}
    for g1 in genes:
        distance_matrix[g1.accession] = {}
        for g2 in genes:
            distance_matrix[g1.accession][g2.accession] = 0
    
    return distance_matrix

In [ ]:
# Print initialised distance matrix
from utilities import print_distance_matrix
dm = init_distance_matrix(ribosome_genes)
print_distance_matrix(dm)


Time to fill in the matrix with distances.

Write a function which calculates the pairwise distance of genes using kmer distance.
You will need to call the 'kmer_distance' function you have written above. 

In [ ]:
# -- GRADED CELL (2 marks) - complete this cell --

def calculate_kmer_distance_matrix(genes, matrix, k):
    ''' 
    Using the initialised distance matrix, our list of genes, and your kmer_distance function,
    populate the matrix with kmer distances between each pair of genes.
    '''
    for g1 in genes:
        id1 = g1.accession
        seq1 = g1.sequence
        for g2 in genes:
            id2 = g2.accession
            seq2 = g2.sequence
            matrix[id1][id2] = kmer_distance(seq1, seq2, k)
    
    return matrix


In [ ]:
matrix = init_distance_matrix(ribosome_genes)
calculate_kmer_distance_matrix(ribosome_genes, matrix, 20)


Let's do the same as above, but this time use the Smith Waterman alignment distance function you wrote. 

In [ ]:
# -- GRADED CELL (1 marks) - complete this cell --

def calculate_sw_alignment_distance_matrix(genes, matrix):
    '''
        Using the initialised distance matrix, our list of genes, and your smith_waterman function,
        populate the matrix with smith-waterman alignment scores between each pair of genes. 
    '''
    for g1 in genes:
        id1 = g1.accession
        seq1 = g1.sequence
        for g2 in genes:
            id2 = g2.accession
            seq2 = g2.sequence
            matrix[id1][id2] = smith_waterman(seq1, seq2)
    
    return matrix

In [ ]:
matrix = init_distance_matrix(ribosome_genes)
calculate_sw_alignment_distance_matrix(ribosome_genes, matrix)


Let's test them out.
The two cells below will use your `calculate_kmer_distance_matrix()`, and `calculate_sw_alignment_distance_matrix()` functions to add distances to the matrix.

**NOTE:** the smith-waterman distance calculations can take time. Give it a minute.

In [ ]:
%%time

distance_matrix = init_distance_matrix(ribosome_genes)
kmer_distance_matrix = calculate_kmer_distance_matrix(
    ribosome_genes,
    distance_matrix,
    8)
print('\nkmer distance matrix')
print_distance_matrix(kmer_distance_matrix)

In [ ]:
%%time

distance_matrix = init_distance_matrix(ribosome_genes)
sw_alignment_distance_matrix = calculate_sw_alignment_distance_matrix(
    ribosome_genes,
    distance_matrix)
print('\nsmith waterman alignment score distance matrix')
print_distance_matrix(sw_alignment_distance_matrix)

Let's visualise those in a better manner for human eyes.
The cell below will plot heatmaps instead of raw numbers.

In [ ]:
from utilities import heatmap
heatmap(kmer_distance_matrix,
        sw_alignment_distance_matrix)

## Short answer question

Answer in the cell below.

You are screening sequencing reads from a mixed sample against a database to try to find out which organism each read came from. Your dataset may be short reads or long reads. Short sequencing reads are accurate, with sequencing errors occurring rarely (less than one error per 1000 b). Long read sequencing can be more error prone, with 1–10 errors per 100 b.

How would the error profiles of different sequencing techniques (short *vs.* long reads) influence your choice between Smith-Waterman and *k*-mer based distance algorithms to compare each read to sequences in the database?

**8 marks**, maximum of 150 words

For error-prone long-read datasets, the errors will negatively influence the effectiveness of k-mer based algorithms more than the Smith-Waterman based algorithms. This is because for k-mer algorithms with a large k value, even a single missequenced nucleotide will greatly and falsely increase the number of unique kmers and hence the distance.  If with a small k value, k-mer based algorithm will not be greatly influenced by errors, but a small k value can lead to low specificity and loss of information. In contrast, Smith-Waterman based algorithms will impose a penalty on sequencing errors. This will not influence the effectiveness as much as it does k-mer based algorithms.  
For accurate short-read datasets, k-mer based algorithms and Smith-Waterman based algorithms will both function well. 


# SECTION 2: CLUSTERING

From the heatmaps, it seems like there are a few clusters in the data. <br>
First, let's convert the pairwise distances to 2D coordinates. 
This is possible using Multidimensional scaling (MDS - which we will cover in a few weeks during lectures).
After we have transformed the distance matrix to 2D coordinates, we can plot it to see if any clusters are evident.

In [ ]:
from utilities import mds_scatterplot, distance_matrix_to_coordinates_MDS

kmer_distances_xy = distance_matrix_to_coordinates_MDS(kmer_distance_matrix)
sw_distances_xy = distance_matrix_to_coordinates_MDS(sw_alignment_distance_matrix)
mds_scatterplot(kmer_distances_xy)
mds_scatterplot(sw_distances_xy)

Seems like there is some clustering happening. <br>
Let's use some clustering algorithms to define the clusters. 
in this manner, we can have an objective way to talk about the patterns in the data.


## TASK 4: DBSCAN (7 points)

We are going to use DBSCAN to cluster the data.

### Description of DBSCAN process


DBSCAN is a clustering method which assigns points to clusters based on continuous regions
of point density above a given threshold. 

We will use the variables 'epsilon' and min_pts to determine if a point belongs to a local 
high-density region. 


Here is a pseudocode outline of the DBSCAN algorithm:


1. For each point in the dataset, we:  	
  a. Compute the epsilon neighbourhood of the point;  
  b. Classify the point as core, border or noise, depending on the number of neighbours.  

2. For each **core** point, *x*, that is **not** assigned a cluster id, we explore the points in the epsilon neighbourhood as follows:  
  a. Assign the point *x* the current cluster id;  
  b. Move to the next point in the epsilon neighbourhood, *y*, and assign the same cluster id.  	
    - If *y* is also a core point, we repeat step 2b to explore the neighbouring points of *y*.  
	- If *y* is not a core point, we move on to the next neighbour of *x* and repeat step 2.


3. Step 2 is repeated until all core points are assigned a cluster id.



In [ ]:
from utilities import euclidean_distance, plot_dbscan
from sklearn.datasets import make_moons

In [ ]:
# Generate curved testing data
x, label = make_moons(n_samples=200, noise=0.1, random_state=19)
# Add background noise
background = np.random.uniform(low=[-1,0],high=[2,2],size=(30,2))
# Join datasets
points_array = np.concatenate([x,background])
# Convert array to list of tuples
data = [tuple(p) for p in points_array]


In [ ]:
# Inspect data
x_values = [x for (x,y) in data]
y_values = [y for (x,y) in data]
plt.scatter(x_values,y_values)

In [ ]:
# -- GRADED CELL (2 marks) - complete this cell --

# Calculating neighbours
# Use provided function euclidean_distance() to find distance between points.

def _get_neighbours(points, p, eps):
    ''' 
        Find all data points that occur within eps distance of point p.
        Return as list of tuples.
    '''
    neighbours = []
    for point in points:
        distance = euclidean_distance(point, p)
        if distance <= eps:
            neighbours.append(point)
    
    return neighbours



In [ ]:
# Tests
assert len(_get_neighbours(points=[(1,1),(1.1,1.1),(5,5)], p=(5,5), eps=0.15)) == 1
assert len(_get_neighbours(points=[(1,1),(1.1,1.1),(5,5)], p=(1,1), eps=0.15)) == 2

In [ ]:
# -- GRADED CELL (2 marks) - complete this cell --
# Labelling point types

def _set_point_labels(points, min_pts, eps):
    '''
        Set point lable to: 
        - 'core' if within eps distance of min_pts number of neighbours
        - 'border' if within eps distance of a core point
        - 'noise' if not within eps distance of a core point
        Input:
        - A list of datapoints stored as tuples of form (x,y)
        - min_pts = minimum number of neighbours for point to be considered 'core'
        - eps = epsilon distance 
        Output:
        - A list of labels in corresponding order to data point list.
    '''
    labels = ['undefined' for point in points ]
    length = len(points)
    lst_core = []
    
    for a in range(0,length):
        p = points[a]
        neighbours = _get_neighbours(points, p, eps)
        neigh_num = len(neighbours)
        if neigh_num >= min_pts:
            labels[a] = 'core'
            lst_core.append(p)    
    for b in range(0,length):
        if labels[b] == 'undefined':
            p_undefined = points[b]
            for c in lst_core:
                distance = euclidean_distance(p_undefined, c)
                if distance <= eps:
                    labels[b] = 'border'
                else:
                    labels[b] = 'noise'
    
    return labels


In [ ]:
assert _set_point_labels([(1,1.01),(1.01,1.01),(1.1,1.1),(5,5)], min_pts=3, eps=0.15) == ['core', 'core', 'core', 'noise']



In [ ]:
# -- GRADED CELL (4 marks) - complete this cell --

def _assign_clusters(data, labels, eps, min_pts):
    '''
       Assign data points to clusters where all points belonging to an individual cluster 
       are reachable from at least one 'core' point in that cluster.
       Input:
        - A list of datapoints stored as tuples of form (x,y)
        - A corresponding list of labels with values 'core', 'border', or 'noise'
        Output:
        - A list of cluster assignments corresponding to input list data
    '''
    def process_neighs(dic, p, idx):
        for p in dic[p]['neigh']:
            if not dic[p]['clusters']:
                dic[p]['clusters'] = idx
                if dic[p]['label'] == 'core':
                    process_neighs(dic, p, idx)
    
    
    clusters = [None for point in data]
    dic = {}
    length = len(data)
    
    for a in range(0,length):
        p = data[a]
        dic[p] = {}
        lst_neigh = _get_neighbours(data, p, eps)
        dic[p]['neigh'] = lst_neigh
        dic[p]['label'] = labels[a]
        dic[p]['clusters'] = clusters[a]
        
    idx = 1
    for p in data:
        if dic[p]['label'] == 'core':
            if not dic[p]['clusters']:
                dic[p]['clusters'] = idx
                process_neighs(dic, p, idx)
                idx += 1
    clusters_lst = []
    for sub_dic in dic.values():
        clusters_lst.append(sub_dic['clusters'])
    clusters = clusters_lst  
    return clusters


In [ ]:
# Basic check
t_data = [(0.9,0.9),(1.01,1),(1.01,1.01),(1,1.01),(10,10.01),(10.01,10),(10.11,10),(10,10.11)]
t_labels = _set_point_labels(t_data, min_pts=4, eps=0.15)
assert len(set(_assign_clusters(t_data, t_labels, min_pts=4, eps=0.15))) == 2, 'Incorrect number of clusters'

In [ ]:
# -- GRADED CELL (2 marks) - complete this cell --

# Put it all together

def dbscan(data, eps=0.25, min_pts=10):
    '''
        Take input list of data points. 
        Return list of data points AND list of cluster assignments.
    '''
    if type(data) == np.ndarray:
        dt = data.tolist()
        data = []
        for p in dt:
            data.append(tuple(p)) 
    labels = _set_point_labels(data, min_pts, eps)
    clusters = _assign_clusters(data, labels, eps, min_pts)
    points = data    
    
    return points, clusters

In [ ]:
%%time
points, clusters = dbscan(data, eps=0.25, min_pts=10)

assert type(points) == type(clusters), 'dbscan function must return a list of data points and a list of cluster assignments'


In [ ]:
%%time
# Run dbscan
points, clusters = dbscan(data, eps=0.23, min_pts=10)

# Plot clusters
plot_dbscan(points, clusters)



## Comparing K-means and DBSCAN clustering

Below we will colour our MDS plots using cluster labels from either KMeans or your implementation of DBSCAN.

In [ ]:
# Lets look at the MDS plot of our Smith-Waterman distances with different clustering methods

# Import KMeans fuction from sklearn
from sklearn.cluster import KMeans
from utilities import mds_scatterplot_clusters

# Cluster with DBSCAN
sw_points, sw_db_clusters = dbscan(sw_distances_xy, eps=20, min_pts=3)

# Cluster with K-means
sw_kmeans = KMeans(n_clusters=3, random_state=0).fit(sw_distances_xy)


# Plot with clusters
print("DBSCAN Clusters")
mds_scatterplot_clusters(sw_points, labels=sw_db_clusters)
print("KMeans Clusters")
mds_scatterplot_clusters(sw_points, labels=sw_kmeans.labels_)

## Short-answer questions

Answer in the cell below.

List 2 benefits and 2 drawbacks of DBSCAN, compared to K-means clustering.

**8 marks**

Benefits:
1 DBSCAN can cluster data into arbitrarily-shaped clusters, while k-means cannot
2 DBSCAN can discriminate outliers and hence is robust to noise
Drawbacks:
1 DBSCAN cannot handle clusters with varied density well
2 DBSCAN is sensitive to parameter values(eps and minPts), which means improper choice of parameter values can greatly influence the effectiveness of DBSCAN 

Answer in the cell below.

We've discussed how DBSCAN can have difficulty identifying clusters with different densities. How would you write an algorithmic approach to use DBSCAN to identify clusters with a range of densities? Describe your approach in plan English, not code.

**4 marks**, maximum of 100 words

Rather than a single parameter eps value, we can design an algorithm that can detect and apply different eps values in different clusters, based on the local density of the initial point in each cluster. (Small eps value for dense clusters, greater for sparse clusters.) First, for each data point, a local density is calculated as the sum of distances from the k-nearest neighbors to the current point. Secondly, data points are descendingly sorted according to local density. Thirdly, we perform DBSCAN cluster assigning, starting with the point with the highest local density towards the point with the lowest. 